# Hyperparameters and Model Validation

We saw earlier the buy-n-large framework of implementing a supervised Machine Learning model.

1. Choose a class of model
2. Choose model hyperparameters
3. Fit the model to the training data
4. Use the model to predict labels for new data

A huge part of this whole process is choosing the model, tuning the parameters of the model and then training and testing of the trained model. Let's have a deeper look into this.